# Importing Libraries

In [140]:
import pandas as pd
import datetime

from varname import nameof
import time

import numpy as np

# Functions for Calculating Technical Indicators

In [141]:
#function to get the stock name

def variable_to_str(df): 
    variable_name = list(df.keys())[1] 
    return variable_name[6:]

In [142]:
variable_to_str(GRAE)

'GRAE'

In [143]:
#Moving Average Convergence Divergence (MACD)

#https://aleksandarhaber.com/macd-moving-average-convergence-divergence-of-stock-price-time-series-in-pandas-and-python/


def MACD(data):
    
    short_period = 12
    long_period  = 26
    signal_period = 9

    stock_name = variable_to_str(df=data)
    
    data["ewm_short"] = data["Price_"+stock_name].ewm(span=short_period, adjust=False).mean()
    data["ewm_long"] = data["Price_"+stock_name].ewm(span=long_period, adjust=False).mean()
    data["MACD"] = data["ewm_short"] - data["ewm_long"]
    data["signal_MACD"] = data["MACD"].ewm(span=signal_period, adjust=False).mean()
    data["bars"] = data["MACD"] - data["signal_MACD"]
    
    data.drop(["ewm_short","ewm_long","signal_MACD","bars"],axis=1,inplace=True)
    
    data.rename(columns={"MACD":"MACD_"+stock_name},inplace=True)
    
    

In [144]:
#Relative Strength Index (RSI)

#https://www.youtube.com/watch?v=2H2Gd7zYVA8

def RSI(data):
    
    stock_name = variable_to_str(df=data)
    
    data["delta"] = data["Price_"+stock_name].diff()
    data["up"] = data["delta"].clip(lower=0)
    data["down"] = -1 * data["delta"].clip(upper=0)
    
    data["ema_up"] = data["up"].ewm(com=13, adjust=False).mean()
    data["ema_down"] = data["down"].ewm(com=13, adjust=False).mean()
    
    data["rs"] = data["ema_up"] / data["ema_down"]
    data["RSI"] = 100 - (100 / (1 + data["rs"]))
    
    data.drop(["delta","up","down","ema_up","ema_down","rs"], axis=1, inplace=True)
    data.rename(columns = {"RSI":"RSI_"+stock_name}, inplace=True)
    

In [145]:
#Commodity Channel Index (CCI)

#https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/cci
#https://gist.github.com/quantra-go-algo/1b37bfb74d69148f0dfbdb5a2c7bdb25

#TP = Typical Price
#sma = simple moving average
#mad = mean deviation

def CCI(data, number_of_days):
    
    stock_name = variable_to_str(df=data)
    
    data["TP"] = (data["High_"+stock_name] + data["Low_"+stock_name] + data["Price_"+stock_name])/3
    data['sma'] = data['TP'].rolling(number_of_days).mean()
    data['mad'] = data['TP'].rolling(number_of_days).apply(lambda x: pd.Series(x).mad())
    data['CCI'] = (data['TP'] - data['sma']) / (0.015 * data['mad']) 
    
    data.drop(["TP","sma", "mad"], axis=1, inplace=True)
    data.rename(columns = {"CCI":"CCI_"+stock_name}, inplace=True)


In [146]:
# Average Directional Index (ADX)

#https://medium.com/codex/algorithmic-trading-with-average-directional-index-in-python-2b5a20ecf06a

def calculateValue(df, column, index):
    previous_value = df.loc[index-1, column]
    current_value = df.loc[index, column]
    return current_value, previous_value

def calculateDM(df, index, stock_name):
    current_high, previous_high = calculateValue(df, "High_"+stock_name, index)
    current_low, previous_low = calculateValue(df, "Low_"+stock_name, index)
    
    dm_positive = current_high - previous_high
    dm_negative = current_low - previous_low
    
    if dm_positive > dm_negative:
        if dm_positive < 0:
            dm_positive = 0.00
        dm_negative = 0.00
        return dm_positive, dm_negative

    elif dm_positive < dm_negative:
        if dm_negative < 0:
            dm_negative = 0.00
        dm_positive = 0.00
        return dm_positive, dm_negative
    
    else:
        if dm_positive < 0:
            dm_positive = 0.00
        dm_negative = 0.00
        return dm_positive, dm_negative

def calculateTR(df, index, stock_name):
    current_high, previous_high = calculateValue(df, "High_"+stock_name, index)
    current_low, previous_low = calculateValue(df, "Low_"+stock_name, index)
    current_close, previous_close = calculateValue(df, "Price_"+stock_name, index)
    ranges = [current_high - current_low, abs(current_high - previous_close), abs(current_low - previous_close)]
    TR = max(ranges)
    return TR

def calculate_first_14(df, index, column):
    result = 0
    for i in range(index-13, index+1):
        result += df.loc[i, column]
    return result

def calculate_subsequent_14(df, index, column):
    return(df.loc[index-1, column+'14'] - (df.loc[index-1, column+'14']/14) + df.loc[index, column])


def calculate_first_adx(df, index):
    result = 0
    for i in range(index-13, index+1):
        result += df.loc[i, 'DX']
    return(result/14)

def calculate_adx(df, index):
    return(round(((df.loc[index-1, 'ADX']*13) + df.loc[index, 'DX'])/14, 2))

def ADX(data):
    
    stock_name = variable_to_str(df=data)
    
    for i in range(1, len(data)):
        dm_pos, dm_neg = calculateDM(data, i, stock_name)
        TR = calculateTR(data, i, stock_name)
        data.loc[i, '+DM'] = dm_pos
        data.loc[i, '-DM'] = dm_neg
        data.loc[i, 'TR'] = TR

        if data.TR.count() == 14:
            data.loc[i, 'TR14'] = calculate_first_14(data, i, 'TR')
            data.loc[i, '+DM14'] = calculate_first_14(data, i, '+DM')
            data.loc[i, '-DM14'] = calculate_first_14(data, i, '-DM')

        elif data.TR.count() >= 14:
            data.loc[i, 'TR14'] = round(calculate_subsequent_14(data, i, 'TR'),2)
            data.loc[i, '+DM14'] = round(calculate_subsequent_14(data, i, '+DM'), 2)
            data.loc[i, '-DM14'] = round(calculate_subsequent_14(data, i, '-DM'), 2)

        if 'TR14' in data.columns:
            data.loc[i, '+DI'] = round((data.loc[i, '+DM14'] / data.loc[i, 'TR14'])*100, 2)
            data.loc[i, '-DI'] = round((data.loc[i, '-DM14'] / data.loc[i, 'TR14'])*100, 2)

            data.loc[i, 'DX'] = round((abs(data.loc[i, '+DI'] - data.loc[i, '-DI'])/abs(data.loc[i, '+DI'] + data.loc[i, '-DI']) )*100 , 2)

        if 'DX' in data.columns:
            if data.DX.count() == 14:
                data.loc[i, 'ADX'] = calculate_first_adx(data, i)

            elif data.DX.count() >= 14:
                data.loc[i, 'ADX'] = calculate_adx(data, i)
    
    data.drop(["+DM","-DM","TR","TR14","+DM14","-DM14","+DI","-DI","DX"], axis=1, inplace=True)
    data.rename(columns={"ADX":"ADX_"+stock_name},inplace=True)


# GRAE - Grameenphone Ltd

In [163]:
GRAE = pd.read_csv("GRAE Historical Data.csv")
len_grae = len(GRAE)
print(len_grae)
GRAE.head()

3017


,Date,Price,Open,High,Low,Vol.,Change %
0,"Sep 04, 2022",288.7,295.0,295.8,288.0,121.84K,-0.45%
1,"Sep 01, 2022",290.0,290.0,291.0,289.0,161.03K,0.76%
2,"Aug 31, 2022",287.8,295.7,295.7,287.0,167.84K,-0.10%
3,"Aug 30, 2022",288.1,295.2,295.3,287.8,148.11K,-1.74%
4,"Aug 29, 2022",293.2,290.0,294.1,288.9,205.84K,1.52%


In [164]:
GRAE.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3012,"Nov 24, 2009",174.8,170.0,177.0,168.8,1.32M,3.74%
3013,"Nov 23, 2009",168.5,164.1,170.0,164.1,1.09M,1.69%
3014,"Nov 22, 2009",165.7,167.0,168.9,163.0,872.40K,1.04%
3015,"Nov 19, 2009",164.0,158.1,167.9,152.0,1.63M,3.67%
3016,"Nov 18, 2009",158.2,170.1,170.2,158.0,2.73M,-7.54%


In [165]:
GRAE = GRAE.iloc[::-1]
GRAE.reset_index(drop=True, inplace=True)

In [166]:
GRAE.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Nov 18, 2009",158.2,170.1,170.2,158.0,2.73M,-7.54%
1,"Nov 19, 2009",164.0,158.1,167.9,152.0,1.63M,3.67%
2,"Nov 22, 2009",165.7,167.0,168.9,163.0,872.40K,1.04%
3,"Nov 23, 2009",168.5,164.1,170.0,164.1,1.09M,1.69%
4,"Nov 24, 2009",174.8,170.0,177.0,168.8,1.32M,3.74%


In [167]:
GRAE.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3012,"Aug 29, 2022",293.2,290.0,294.1,288.9,205.84K,1.52%
3013,"Aug 30, 2022",288.1,295.2,295.3,287.8,148.11K,-1.74%
3014,"Aug 31, 2022",287.8,295.7,295.7,287.0,167.84K,-0.10%
3015,"Sep 01, 2022",290.0,290.0,291.0,289.0,161.03K,0.76%
3016,"Sep 04, 2022",288.7,295.0,295.8,288.0,121.84K,-0.45%


In [168]:
GRAE.rename(columns={"Price":"Price_GRAE","Open":"Open_GRAE","High":"High_GRAE","Low":"Low_GRAE","Vol.":"Vol_GRAE","Change %":"Change_GRAE"},inplace=True)
GRAE.columns

Index(['Date', 'Price_GRAE', 'Open_GRAE', 'High_GRAE', 'Low_GRAE', 'Vol_GRAE',
       'Change_GRAE'],
      dtype='object')

In [169]:
GRAE.dtypes

Date            object
Price_GRAE     float64
Open_GRAE      float64
High_GRAE      float64
Low_GRAE       float64
Vol_GRAE        object
Change_GRAE     object
dtype: object

In [170]:
GRAE["Date"] = pd.to_datetime(GRAE["Date"])
GRAE.dtypes

Date           datetime64[ns]
Price_GRAE            float64
Open_GRAE             float64
High_GRAE             float64
Low_GRAE              float64
Vol_GRAE               object
Change_GRAE            object
dtype: object

In [171]:
print(GRAE.isna().sum().sum())
GRAE[GRAE.isna().any(axis=1)]

2


,Date,Price_GRAE,Open_GRAE,High_GRAE,Low_GRAE,Vol_GRAE,Change_GRAE
2077,2018-08-05,389.4,389.4,389.4,389.4,NaN,0.00%
2207,2019-02-17,389.9,389.9,389.9,389.9,NaN,0.00%


In [173]:
GRAE["Vol_GRAE"].bfill(limit=1, inplace=True)
print(GRAE.isna().sum().sum())
GRAE.reset_index(drop=True, inplace=True)

0


In [174]:
#Vol
GRAE["Vol_GRAE"] = GRAE["Vol_GRAE"].astype("str")
print(GRAE["Vol_GRAE"].apply(lambda x:x[-1:]).value_counts())
units = {'K':1e3, 'M':1e6}
GRAE["Vol_GRAE"] = GRAE["Vol_GRAE"].apply(lambda x:float(x[:-1])*units[x[-1:]])


#Change
GRAE["Change_GRAE"] = GRAE["Change_GRAE"].apply(lambda x:float(x[:-1]))

K    2700
M     317
Name: Vol_GRAE, dtype: int64


In [175]:
%%time

#calculating MACD
MACD(GRAE)

#calculating RSI
RSI(GRAE)

#calculating CCI
CCI(GRAE, number_of_days = 14)

#calculating RSI
ADX(GRAE)

CPU times: total: 3.64 s
Wall time: 3.65 s


In [176]:
GRAE.drop(["Open_GRAE", "High_GRAE", "Low_GRAE", "Change_GRAE"], axis=1, inplace=True)
GRAE.head()

,Date,Price_GRAE,Vol_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE
0,2009-11-18,158.2,2730000.0,0.000000,NaN,NaN,NaN
1,2009-11-19,164.0,1630000.0,0.462678,100.0,NaN,NaN
2,2009-11-22,165.7,872400.0,0.955515,100.0,NaN,NaN
3,2009-11-23,168.5,1090000.0,1.554114,100.0,NaN,NaN
4,2009-11-24,174.8,1320000.0,2.507954,100.0,NaN,NaN


In [177]:
GRAE.tail()

,Date,Price_GRAE,Vol_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE
3012,2022-08-29,293.2,205840.0,-1.415638,53.992098,18.577495,12.25
3013,2022-08-30,288.1,148110.0,-1.430574,45.069171,3.841787,13.72
3014,2022-08-31,287.8,167840.0,-1.449905,44.602222,19.481341,15.17
3015,2022-09-01,290.0,161030.0,-1.273028,48.792203,76.054852,15.60
3016,2022-09-04,288.7,121840.0,-1.223646,46.551626,97.438272,17.12


In [178]:
GRAE.isna().sum()

Date           0
Price_GRAE     0
Vol_GRAE       0
MACD_GRAE      0
RSI_GRAE       1
CCI_GRAE      13
ADX_GRAE      27
dtype: int64

In [179]:
GRAE["RSI_GRAE"].bfill(limit=1, inplace=True)
GRAE["CCI_GRAE"].bfill(limit=13, inplace=True)
GRAE["ADX_GRAE"].bfill(limit=27, inplace=True)

In [180]:
GRAE.isna().sum()

Date          0
Price_GRAE    0
Vol_GRAE      0
MACD_GRAE     0
RSI_GRAE      0
CCI_GRAE      0
ADX_GRAE      0
dtype: int64

In [181]:
GRAE.dtypes

Date          datetime64[ns]
Price_GRAE           float64
Vol_GRAE             float64
MACD_GRAE            float64
RSI_GRAE             float64
CCI_GRAE             float64
ADX_GRAE             float64
dtype: object

In [182]:
GRAE.describe()

,Price_GRAE,Vol_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE
count,3017.000000,3.017000e+03,3017.000000,3017.000000,3017.000000,3017.000000
mean,288.995360,4.597769e+05,0.308959,50.736245,-0.696807,19.727504
std,84.910065,5.782132e+05,6.409894,14.086098,111.360248,9.457826
min,139.100000,1.412000e+04,-21.608264,8.093966,-416.427932,5.840000
25%,214.700000,1.360600e+05,-3.469123,41.102278,-84.520357,12.770000
50%,295.100000,2.738000e+05,-0.132546,49.252931,-8.547009,17.900000
75%,349.900000,5.498000e+05,3.348015,59.754584,80.174402,24.250000
max,510.700000,7.490000e+06,36.922365,100.000000,466.666667,58.660000


# SQPH - Square Pharmaceuticals Ltd 

In [183]:
SQPH = pd.read_csv("SQPH Historical Data.csv")
len_sqph = len(SQPH)
print(len_sqph)
SQPH.head()

3504


,Date,Price,Open,High,Low,Vol.,Change %
0,"Sep 04, 2022",212.6,211.8,213.5,211.5,276.44K,0.38%
1,"Sep 01, 2022",211.8,213.0,213.0,211.4,1.02M,0.33%
2,"Aug 31, 2022",211.1,212.0,213.0,209.9,892.16K,-0.38%
3,"Aug 30, 2022",211.9,213.0,213.0,211.5,679.86K,-0.09%
4,"Aug 29, 2022",212.1,214.0,214.0,211.5,764.64K,-0.52%


In [184]:
SQPH.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3499,"Dec 02, 2007",46.04,47.69,47.69,45.30,1.13M,-3.01%
3500,"Nov 29, 2007",47.47,47.40,47.63,47.30,794.85K,0.02%
3501,"Nov 28, 2007",47.46,47.69,47.76,47.35,819.61K,-0.52%
3502,"Nov 27, 2007",47.71,48.76,48.76,47.68,795.76K,-1.55%
3503,"Nov 26, 2007",48.46,48.19,49.19,48.05,598.48K,1.08%


In [185]:
SQPH = SQPH.iloc[::-1]
SQPH.reset_index(drop=True, inplace=True)
SQPH.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Nov 26, 2007",48.46,48.19,49.19,48.05,598.48K,1.08%
1,"Nov 27, 2007",47.71,48.76,48.76,47.68,795.76K,-1.55%
2,"Nov 28, 2007",47.46,47.69,47.76,47.35,819.61K,-0.52%
3,"Nov 29, 2007",47.47,47.40,47.63,47.30,794.85K,0.02%
4,"Dec 02, 2007",46.04,47.69,47.69,45.30,1.13M,-3.01%


In [186]:
SQPH.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3499,"Aug 29, 2022",212.1,214.0,214.0,211.5,764.64K,-0.52%
3500,"Aug 30, 2022",211.9,213.0,213.0,211.5,679.86K,-0.09%
3501,"Aug 31, 2022",211.1,212.0,213.0,209.9,892.16K,-0.38%
3502,"Sep 01, 2022",211.8,213.0,213.0,211.4,1.02M,0.33%
3503,"Sep 04, 2022",212.6,211.8,213.5,211.5,276.44K,0.38%


In [187]:
SQPH.rename(columns={"Price":"Price_SQPH","Open":"Open_SQPH","High":"High_SQPH","Low":"Low_SQPH","Vol.":"Vol_SQPH","Change %":"Change_SQPH"},inplace=True)
SQPH.columns

Index(['Date', 'Price_SQPH', 'Open_SQPH', 'High_SQPH', 'Low_SQPH', 'Vol_SQPH',
       'Change_SQPH'],
      dtype='object')

In [188]:
SQPH.dtypes

Date            object
Price_SQPH     float64
Open_SQPH      float64
High_SQPH      float64
Low_SQPH       float64
Vol_SQPH        object
Change_SQPH     object
dtype: object

In [189]:
SQPH["Date"] = pd.to_datetime(SQPH["Date"])
SQPH.dtypes

Date           datetime64[ns]
Price_SQPH            float64
Open_SQPH             float64
High_SQPH             float64
Low_SQPH              float64
Vol_SQPH               object
Change_SQPH            object
dtype: object

In [103]:
print(SQPH.isna().sum().sum())
SQPH[SQPH.isna().any(axis=1)]

1


,Date,Price_SQPH,Open_SQPH,High_SQPH,Low_SQPH,Vol_SQPH,Change_SQPH
3417,2008-04-06,62.18,62.18,62.18,62.18,NaN,0.00%


In [190]:
SQPH["Vol_SQPH"].bfill(limit=1, inplace=True)
print(SQPH.isna().sum().sum())
SQPH.reset_index(drop=True, inplace=True)

0


In [191]:
#Vol
SQPH["Vol_SQPH"] = SQPH["Vol_SQPH"].astype("str")
SQPH["Vol_SQPH"].apply(lambda x:x[-1:]).value_counts()
units = {'K':1e3, 'M':1e6}
SQPH["Vol_SQPH"] = SQPH["Vol_SQPH"].apply(lambda x:float(x[:-1])*units[x[-1:]])

#Change
SQPH["Change_SQPH"] = SQPH["Change_SQPH"].apply(lambda x:float(x[:-1]))

In [192]:
%%time

#calculating MACD
MACD(SQPH)

#calculating RSI
RSI(SQPH)

#calculating CCI
CCI(SQPH, number_of_days = 14)

#calculating RSI
ADX(SQPH)

CPU times: total: 4.38 s
Wall time: 4.39 s


In [193]:
SQPH.drop(["Open_SQPH", "High_SQPH", "Low_SQPH", "Change_SQPH"], axis=1, inplace=True)
SQPH.head()

,Date,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
0,2007-11-26,48.46,598480.0,0.000000,NaN,NaN,NaN
1,2007-11-27,47.71,795760.0,-0.059829,0.000000,NaN,NaN
2,2007-11-28,47.46,819610.0,-0.125965,0.000000,NaN,NaN
3,2007-11-29,47.47,794850.0,-0.175547,0.107576,NaN,NaN
4,2007-12-02,46.04,1130000.0,-0.326468,0.092287,NaN,NaN


In [194]:
SQPH.isna().sum()

Date           0
Price_SQPH     0
Vol_SQPH       0
MACD_SQPH      0
RSI_SQPH       1
CCI_SQPH      19
ADX_SQPH      27
dtype: int64

In [195]:
SQPH["RSI_SQPH"].bfill(limit=1, inplace=True)
SQPH["CCI_SQPH"].bfill(limit=19, inplace=True)
SQPH["ADX_SQPH"].bfill(limit=27, inplace=True)

In [196]:
SQPH.isna().sum()

Date          0
Price_SQPH    0
Vol_SQPH      0
MACD_SQPH     0
RSI_SQPH      0
CCI_SQPH      0
ADX_SQPH      0
dtype: int64

In [197]:
SQPH.head()

,Date,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
0,2007-11-26,48.46,598480.0,0.000000,0.000000,-36.783439,27.256429
1,2007-11-27,47.71,795760.0,-0.059829,0.000000,-36.783439,27.256429
2,2007-11-28,47.46,819610.0,-0.125965,0.000000,-36.783439,27.256429
3,2007-11-29,47.47,794850.0,-0.175547,0.107576,-36.783439,27.256429
4,2007-12-02,46.04,1130000.0,-0.326468,0.092287,-36.783439,27.256429


In [198]:
SQPH.dtypes

Date          datetime64[ns]
Price_SQPH           float64
Vol_SQPH             float64
MACD_SQPH            float64
RSI_SQPH             float64
CCI_SQPH             float64
ADX_SQPH             float64
dtype: object

In [199]:
SQPH.describe()

,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
count,3504.000000,3.504000e+03,3504.000000,3504.000000,3504.000000,3504.000000
mean,149.669047,9.828234e+05,0.328411,52.206290,inf,18.420186
std,73.003802,1.060261e+06,2.300802,13.962268,NaN,7.903185
min,39.450000,2.430000e+04,-8.641051,0.000000,-392.036043,5.240000
25%,76.057500,3.634200e+05,-0.721743,43.351227,-78.042860,12.320000
50%,168.575000,6.299600e+05,0.249892,51.905897,9.229607,17.095000
75%,215.525000,1.170000e+06,1.307358,60.985602,87.890678,22.770000
max,291.550000,1.288000e+07,9.303017,90.632171,inf,54.430000


In [200]:
SQPH[SQPH["CCI_SQPH"]==np.inf]

,Date,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
2975,2020-06-29,164.29,205780.0,-1.655996,42.811316,inf,32.01


In [201]:
SQPH.replace([np.inf, -np.inf], np.nan, inplace=True)
SQPH[SQPH["CCI_SQPH"]==np.inf]

,Date,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH


In [202]:
SQPH.isna().sum()

Date          0
Price_SQPH    0
Vol_SQPH      0
MACD_SQPH     0
RSI_SQPH      0
CCI_SQPH      1
ADX_SQPH      0
dtype: int64

In [203]:
SQPH["CCI_SQPH"].bfill(limit=1, inplace=True)
SQPH.isna().sum()

Date          0
Price_SQPH    0
Vol_SQPH      0
MACD_SQPH     0
RSI_SQPH      0
CCI_SQPH      0
ADX_SQPH      0
dtype: int64

In [204]:
SQPH.describe()

,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH
count,3504.000000,3.504000e+03,3504.000000,3504.000000,3504.000000,3504.000000
mean,149.669047,9.828234e+05,0.328411,52.206290,9.905156,18.420186
std,73.003802,1.060261e+06,2.300802,13.962268,114.355334,7.903185
min,39.450000,2.430000e+04,-8.641051,0.000000,-392.036043,5.240000
25%,76.057500,3.634200e+05,-0.721743,43.351227,-78.042860,12.320000
50%,168.575000,6.299600e+05,0.249892,51.905897,9.229607,17.095000
75%,215.525000,1.170000e+06,1.307358,60.985602,87.890678,22.770000
max,291.550000,1.288000e+07,9.303017,90.632171,466.666667,54.430000


# ACIF - ACI Formulations Ltd

In [205]:
ACIF = pd.read_csv("ACIF Historical Data.csv")
len_acif = len(ACIF)
print(len_acif)
ACIF.head()

3268


,Date,Price,Open,High,Low,Vol.,Change %
0,"Sep 04, 2022",173.8,183.0,184.0,171.6,831.80K,-0.57%
1,"Sep 01, 2022",174.8,174.4,175.8,170.9,551.36K,1.92%
2,"Aug 31, 2022",171.5,166.5,174.9,163.1,1.16M,4.38%
3,"Aug 30, 2022",164.3,163.0,166.2,162.8,304.96K,1.05%
4,"Aug 29, 2022",162.6,165.0,166.0,161.5,387.64K,-1.28%


In [206]:
ACIF.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3263,"Nov 27, 2008",82.22,84.66,92.59,80.69,34.40K,-8.05%
3264,"Nov 26, 2008",89.42,91.64,91.64,82.01,4.54K,-6.27%
3265,"Nov 25, 2008",95.40,100.53,100.53,94.71,6.61K,-4.35%
3266,"Nov 24, 2008",99.74,95.24,105.29,95.24,7.37K,-0.88%
3267,"Nov 23, 2008",100.63,106.98,115.87,100.53,37.42K,-5.94%


In [207]:
ACIF = ACIF.iloc[::-1]
ACIF.reset_index(drop=True, inplace=True)
ACIF.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Nov 23, 2008",100.63,106.98,115.87,100.53,37.42K,-5.94%
1,"Nov 24, 2008",99.74,95.24,105.29,95.24,7.37K,-0.88%
2,"Nov 25, 2008",95.40,100.53,100.53,94.71,6.61K,-4.35%
3,"Nov 26, 2008",89.42,91.64,91.64,82.01,4.54K,-6.27%
4,"Nov 27, 2008",82.22,84.66,92.59,80.69,34.40K,-8.05%


In [209]:
ACIF.tail()

,Date,Price,Open,High,Low,Vol.,Change %
3263,"Aug 29, 2022",162.6,165.0,166.0,161.5,387.64K,-1.28%
3264,"Aug 30, 2022",164.3,163.0,166.2,162.8,304.96K,1.05%
3265,"Aug 31, 2022",171.5,166.5,174.9,163.1,1.16M,4.38%
3266,"Sep 01, 2022",174.8,174.4,175.8,170.9,551.36K,1.92%
3267,"Sep 04, 2022",173.8,183.0,184.0,171.6,831.80K,-0.57%


In [210]:
ACIF.rename(columns={"Price":"Price_ACIF","Open":"Open_ACIF","High":"High_ACIF","Low":"Low_ACIF","Vol.":"Vol_ACIF","Change %":"Change_ACIF"},inplace=True)
ACIF.columns

Index(['Date', 'Price_ACIF', 'Open_ACIF', 'High_ACIF', 'Low_ACIF', 'Vol_ACIF',
       'Change_ACIF'],
      dtype='object')

In [211]:
ACIF.dtypes

Date            object
Price_ACIF     float64
Open_ACIF      float64
High_ACIF      float64
Low_ACIF       float64
Vol_ACIF        object
Change_ACIF     object
dtype: object

In [212]:
ACIF["Date"] = pd.to_datetime(ACIF["Date"])
ACIF.dtypes

Date           datetime64[ns]
Price_ACIF            float64
Open_ACIF             float64
High_ACIF             float64
Low_ACIF              float64
Vol_ACIF               object
Change_ACIF            object
dtype: object

In [213]:
print(ACIF.isna().sum().sum())

0


In [214]:
#Vol
ACIF["Vol_ACIF"] = ACIF["Vol_ACIF"].astype("str")
ACIF["Vol_ACIF"].apply(lambda x:x[-1:]).value_counts()
units = {'K':1e3, 'M':1e6}
ACIF["Vol_ACIF"] = ACIF["Vol_ACIF"].apply(lambda x:float(x[:-1])*units[x[-1:]])

#Change
ACIF["Change_ACIF"] = ACIF["Change_ACIF"].apply(lambda x:float(x[:-1]))

In [215]:
%%time

#calculating MACD
MACD(ACIF)

#calculating RSI
RSI(ACIF)

#calculating CCI
CCI(ACIF, number_of_days = 14)

#calculating RSI
ADX(ACIF)

CPU times: total: 4.03 s
Wall time: 4.04 s


In [216]:
ACIF.drop(["Open_ACIF", "High_ACIF", "Low_ACIF", "Change_ACIF"], axis=1, inplace=True)
ACIF.head()

,Date,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
0,2008-11-23,100.63,37420.0,0.000000,NaN,NaN,NaN
1,2008-11-24,99.74,7370.0,-0.070997,0.0,NaN,NaN
2,2008-11-25,95.40,6610.0,-0.472023,0.0,NaN,NaN
3,2008-11-26,89.42,4540.0,-1.257876,0.0,NaN,NaN
4,2008-11-27,82.22,34400.0,-2.433596,0.0,NaN,NaN


In [217]:
ACIF.isna().sum()

Date           0
Price_ACIF     0
Vol_ACIF       0
MACD_ACIF      0
RSI_ACIF       1
CCI_ACIF      13
ADX_ACIF      27
dtype: int64

In [218]:
ACIF["RSI_ACIF"].bfill(limit=1, inplace=True)
ACIF["CCI_ACIF"].bfill(limit=13, inplace=True)
ACIF["ADX_ACIF"].bfill(limit=27, inplace=True)

In [219]:
ACIF.isna().sum()

Date          0
Price_ACIF    0
Vol_ACIF      0
MACD_ACIF     0
RSI_ACIF      0
CCI_ACIF      0
ADX_ACIF      0
dtype: int64

In [220]:
ACIF.head()

,Date,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
0,2008-11-23,100.63,37420.0,0.000000,0.0,46.645149,37.230714
1,2008-11-24,99.74,7370.0,-0.070997,0.0,46.645149,37.230714
2,2008-11-25,95.40,6610.0,-0.472023,0.0,46.645149,37.230714
3,2008-11-26,89.42,4540.0,-1.257876,0.0,46.645149,37.230714
4,2008-11-27,82.22,34400.0,-2.433596,0.0,46.645149,37.230714


In [221]:
ACIF.tail()

,Date,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
3263,2022-08-29,162.6,387640.0,1.382056,54.344686,47.094801,44.25
3264,2022-08-30,164.3,304960.0,1.479702,57.214890,58.023050,44.17
3265,2022-08-31,171.5,1160000.0,2.113701,66.749243,128.540305,45.28
3266,2022-09-01,174.8,551360.0,2.849584,70.044171,163.850364,43.81
3267,2022-09-04,173.8,831800.0,3.313885,67.850027,171.308901,43.51


In [222]:
ACIF.dtypes

Date          datetime64[ns]
Price_ACIF           float64
Vol_ACIF             float64
MACD_ACIF            float64
RSI_ACIF             float64
CCI_ACIF             float64
ADX_ACIF             float64
dtype: object

In [223]:
ACIF.describe()

,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
count,3268.000000,3.268000e+03,3268.000000,3268.000000,3268.000000,3268.000000
mean,124.084819,1.608726e+05,0.128709,49.122412,-7.736273,24.684526
std,41.476406,3.174866e+05,3.403274,11.676005,108.659496,11.362707
min,40.320000,6.900000e+02,-7.263533,0.000000,-362.554357,4.620000
25%,84.760000,2.409000e+04,-1.735724,41.535500,-91.743138,15.647500
50%,119.110000,6.556500e+04,-0.305278,48.078914,-22.024481,22.145000
75%,157.062500,1.730175e+05,1.349173,56.145732,68.965130,32.080000
max,242.860000,8.600000e+06,20.388445,95.534066,421.432330,60.160000


# Merge - 3 Stocks

In [224]:
print(min(len_grae, len_sqph, len_acif))
print(max(len_grae, len_sqph, len_acif))

3017
3504


In [225]:
data = pd.merge(pd.merge(GRAE,SQPH,on='Date'),ACIF,on='Date')
print(len(data))
data.head()

2991


,Date,Price_GRAE,Vol_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
0,2009-11-18,158.2,2730000.0,0.000000,100.0,-5.840677,14.040714,50.47,3310000.0,0.485585,63.804225,92.895442,17.86,109.71,85210.0,-2.088105,35.793253,-82.824393,43.85
1,2009-11-19,164.0,1630000.0,0.462678,100.0,-5.840677,14.040714,50.56,2540000.0,0.525514,64.444751,91.644205,16.70,109.65,108200.0,-2.195135,35.680455,-74.623541,43.29
2,2009-11-22,165.7,872400.0,0.955515,100.0,-5.840677,14.040714,50.46,2160000.0,0.542831,63.108436,81.210533,15.81,110.60,116230.0,-2.178191,38.960434,-48.865595,41.84
3,2009-11-23,168.5,1090000.0,1.554114,100.0,-5.840677,14.040714,51.80,5750000.0,0.657107,71.605138,158.707188,16.71,110.29,124580.0,-2.164822,38.274534,-43.144851,40.52
4,2009-11-24,174.8,1320000.0,2.507954,100.0,-5.840677,14.040714,52.81,3980000.0,0.819721,76.077450,225.934820,15.52,112.57,123160.0,-1.947797,45.828360,8.339818,37.73


In [226]:
data.tail()

,Date,Price_GRAE,Vol_GRAE,MACD_GRAE,RSI_GRAE,CCI_GRAE,ADX_GRAE,Price_SQPH,Vol_SQPH,MACD_SQPH,RSI_SQPH,CCI_SQPH,ADX_SQPH,Price_ACIF,Vol_ACIF,MACD_ACIF,RSI_ACIF,CCI_ACIF,ADX_ACIF
2986,2022-08-29,293.2,205840.0,-1.415638,53.992098,18.577495,12.25,212.1,764640.0,-0.323794,46.565653,-47.804878,12.65,162.6,387640.0,1.382056,54.344686,47.094801,44.25
2987,2022-08-30,288.1,148110.0,-1.430574,45.069171,3.841787,13.72,211.9,679860.0,-0.347485,45.734604,-96.936275,12.13,164.3,304960.0,1.479702,57.214890,58.023050,44.17
2988,2022-08-31,287.8,167840.0,-1.449905,44.602222,19.481341,15.17,211.1,892160.0,-0.425904,42.469598,-184.863124,11.65,171.5,1160000.0,2.113701,66.749243,128.540305,45.28
2989,2022-09-01,290.0,161030.0,-1.273028,48.792203,76.054852,15.60,211.8,1020000.0,-0.426650,46.095819,-57.482442,11.81,174.8,551360.0,2.849584,70.044171,163.850364,43.81
2990,2022-09-04,288.7,121840.0,-1.223646,46.551626,97.438272,17.12,212.6,276440.0,-0.358554,49.976497,13.355780,11.46,173.8,831800.0,3.313885,67.850027,171.308901,43.51


In [227]:
data.columns

Index(['Date', 'Price_GRAE', 'Vol_GRAE', 'MACD_GRAE', 'RSI_GRAE', 'CCI_GRAE',
       'ADX_GRAE', 'Price_SQPH', 'Vol_SQPH', 'MACD_SQPH', 'RSI_SQPH',
       'CCI_SQPH', 'ADX_SQPH', 'Price_ACIF', 'Vol_ACIF', 'MACD_ACIF',
       'RSI_ACIF', 'CCI_ACIF', 'ADX_ACIF'],
      dtype='object')

In [228]:
data.isna().sum().sum()

0

In [229]:
data.to_csv("Stock Data - Including COVID 19 .csv",index=False)